# Explications pas à pas de la résolution d'un problème

Ce notebook est consacré à la génération d'une suite d'explications pour des problèmes de satisfaction de contraintes.

Les deux premières cellules de code sont issues de [la page `github` de cpmy](https://github.com/CPMpy/cpmpy) et la compréhension complète de leur fonctionnement nécessite l'étude des articles 

[Ignatiev, A., Previti, A., Liffiton, M. and Marques-Silva, J (2015). Smallest MUS extraction with minimal hitting set dualization. International Conference on Principles and Practice of Constraint Programming. Springer, Cham, 2015](https://citeseerx.ist.psu.edu/document?repid=rep1&type=pdf&doi=535cb80c82f3b9d4026aef397c5e2c9cc8494ac1)

et

[Bogaerts, B., Gamba, E., & Guns, T. (2021). A framework for step-wise explaining how to solve constraint satisfaction problems. Artificial Intelligence, 300, 103550](https://arxiv.org/pdf/2006.06343.pdf).

On se contentera d'indiquer le principe de ce fonctionnement.

## SMUS

Si $F$ est un ensemble insatisfiable de contraintes, un *MUS* (Minimal Unsatisfiable Subset) est $U\subset F$ insatisfiable et minimal ($\forall c\in U,\, U\setminus \{c\}$ est satisfiable) et un *SMUS* (Smallest MUS) est un MUS de cardinal minimum.

La classe `SMUSAlgo` définie ci-dessous permet de calculer un SMUS d'un ensemble insatisfiable de contraintes.

Pour cela, elle utilise les notions suivantes

Un *CS* (Correction Subset) est un sous ensemble *correcteur*, c'est à dire un $ C\subset  F$ avec $ F\setminus C$ satisfiable, et un *MCS* est un sous ensemble correcteur minimal.  
Un *HS* (Hitting Set) d'un ensemble $\mathcal A$ de parties d'un ensemble $E$ est un ensemble $B\subset E$ *rencontrant* tous les éléments de $\mathcal A$ : $\forall A\in\mathcal A,\,A\cap B\neq\emptyset$.  
Un *MHS* de $\mathcal A$ est, bien sûr, un HS minimal.

On a la propriété de dualité suivante qui se vérifie par des raisonnements ensemblistes élémentaires.

Si $F$ est insatisfiable et $U,C\in\mathcal P(F)$,  
$U$ est un MUS de $F$ $\Leftrightarrow$ $U$ est un MHS de l'ensemble des MCSs de $F$  
$C$ est un MCS de $F$ $\Leftrightarrow$ $C$ est un MHS de l'ensemble des MUSs de $F$

Par suite, un SMUS est un SMHS (plus petit MHS) de l'ensemble des MCSs. 

On peut aussi en déduire que, pour qu'un sous ensemble $U$ soit un SMUS, il suffit qu'il soit insatisfiable et qu'il existe un ensemble de MCSs dont $U$ soit un SMHS.

C'est le principe de l'algorithme de recherche d'un SMUS.

In [9]:
#! pip install cpmpy
from cpmpy import *
from cpmpy.transformations.get_variables import get_variables
import numpy as np

class SMUSAlgo:

    def __init__(self, soft, hard=[], solvername="ortools", hs_solvername="ortools", disable_corr_enum=True):
        """
            Smallest Minimal Unsatisfiable Subsets (SMUS) for CSP explanations [1,2]

            SMUS relies on the implicit hitting set duality between
            MUSes and MCSes for a given formula F [2, 3]:

                - A set S \subseteq of F is an MCS of F iff it is a minimal hitting set
                 of MUSs(F).
                - A set S \subseteq F is a MUS of F iff it is a minimal hitting set
                 of MCSs(F).

            Builds a MIP model for computing minimal (optimal) hitting sets. Repeatedly
            checks for satisfiability until UNSAT, which means an SMUS is found.

            If SAT, and sat solver computes a model. The complement w.r.t F is
            computed and added as a new set-to-hit.

            Args
            ----

                hard : Hard constraints must be included in the MUS

                soft : Soft Constraints can be selected by the MUS algorithm

            [1] Ignatiev, A., Ignatiev, A., Previti, A., Liffiton, M. and Marques-Silva, J (2015).  
            Smallest MUS extraction with minimal hitting set dualization. 
            International Conference on Principles and Practice of Constraint Programming. 
            Springer, Cham, 2015.

            [2] Gamba, E., Bogaerts, B., & Guns, T. (8 2021). Efficiently Explaining CSPs
            with Unsatisfiable Subset Optimization. In Z.-H. Zhou (Red), Proceedings of the
            Thirtieth International Joint Conference on Artificial Intelligence,
            IJCAI-21 (bll 1381–1388). doi:10.24963/ijcai.2021/191.

            [3] Liffiton, M. H., & Sakallah, K. A. (2008). Algorithms for computing minimal
            unsatisfiable subsets of constraints. Journal of Automated Reasoning, 40(1), 1-33.

            [4] Reiter, R. (1987). A theory of diagnosis from first principles.
            Artificial intelligence, 32(1), 57-95.
        """
        
        self.soft = cpm_array(soft)
        self.hard = hard
        
        self.assum_vars = boolvar(len(soft))

        self.solver = SolverLookup.get(solvername)
        self.solver += self.hard
        self.solver += self.assum_vars.implies(self.soft)
        
        self.maxsat_solver = SolverLookup.get(solvername)
        self.maxsat_solver += self.hard
        self.maxsat_solver += self.assum_vars.implies(self.soft)
        self.maxsat_solver.maximize(sum(self.assum_vars))

        # Hitting Set MODEL is described by:
        #     - x_l={0,1} if assumption variable is inside the hitting set (1) or not (0).
        #     - c_lj={0,1} is 1 (0) if the literal l is (not) present in the set-to-hit j.
        # Subject to:
        #     (1) sum(x_l * c_lj) >= 1 for all hitting sets j.
        #         = The hitting set must hit all sets-to-hit.
        self.hs_solver= SolverLookup.get(hs_solvername)
        self.hs_solver += sum(self.assum_vars) >= 1
        # Objective:
        #         min sum(x_l) 
        self.hs_solver.minimize(sum(self.assum_vars))

        self.mus = None
        self.disable_corr_enum = disable_corr_enum
        
    def iterate(self, n=1):
        '''
            SMUS-Core computes n iterations of the algorithm.
        '''
        for _ in range(n):
            assert self.hs_solver.solve()
            h = self.assum_vars.value() == 1
            
            if not self.solver.solve(assumptions=self.assum_vars[h]):
                # UNSAT subset, return
                self.mus = set(self.soft[self.assum_vars.value()])
                return
                
            # Find disjunctive sets
            mcses = self.corr_enum(h)

            for mcs in mcses:
                self.hs_solver += sum(self.assum_vars[mcs]) >= 1

    def corr_enum(self, h):
           
        mcses = []
        hp = np.array(h)

        sat, ss = self.grow(hp)
        
        if self.disable_corr_enum:
            return [~ss]

        while(sat):
            mcs = ~ss
            mcses.append(mcs)
            hp = hp | mcs
            sat, ss = self.grow(hp)

        return mcses            
    
    def grow(self, h):
        sat = self.maxsat_solver.solve(assumptions=self.assum_vars[h])
        return sat, self.assum_vars.value() == 1


## Explication

Soit $M_0$ un modèle satisfiable et admettant une unique solution.  
Une contrainte de $M_0$ de la forme $x == v$ ($x$ variable, $v$ valeur) est appelée un *fait*.  

Avec les arguments 
- le dictionnaire *given* formé des associations $x\,:\,v$ pour tous les faits $x == v$,
- l'ensemble *vars_to_expl* des variables de $M_0$ qui ne sont pas des clés de *given*  
- et les contraintes *constraints* qui ne sont pas des faits.

`split_ocus`(*given*, *vars_to_expl*, *constraints*) renvoie un triplet *explication* $=(E,S,N)$ avec  
$E$ est un ensemble de faits, $S\subset$ *constraints* et $N$ est une liste contenant un unique élément $y==w$ où $y\in$ *vars_to_expl*  
satisfaisant à  

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; ($E$ et $S)\Rightarrow y=w$ &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; (1)

De plus, les ensembles $E$ et $S$ sont aussi petits que possible, de sorte qu'un humain puisse facilement démontrer (1). C'est en cela que le triplet $(E,S,N)$ constitue une explication du fait que la valeur de $y$ est $w$ dans la solution de $M_0$.

Notons que l'on peut de nouveau appliquer `split_ocus` au modèle $M_1$ obtenu en rajoutant à $M_0$ le fait $y==w$ ; on obtient ainsi l'explication d'une nouvelle variable et on peut continuer ce procédé jusqu'à épuisement des variables à expliquer. On aura alors une suite d'explications permettant de résoudre entièrement $M_0$.

Le procédé utilisé par `split_ocus` consiste tout d'abord, pour chaque variable $y\in$ *vars_to_expl*, à calculer (à l'aide de `SMUSAlgo`) un SMUS $U_y$ du modèle insatisfiable $M_0+[y\neq w]$ (où $w$ est la valeur de $y$ dans la solution de $M_0$). $U_y'=U_y\setminus \{y\neq w\}$ est alors une explication minimum de $y=w$. Il reste à choisir $y$ pour que $U_y'$ soit minimum 



In [126]:
def split_ocus(given, vars_to_expl, constraints, verbose=False):
    """
        Split ocus first propagates the given variable-value assignments and 
        constraints to find a satisfying solution. 
        split_ocus keep track of a priority queue (PQ) of SMUS extractors 
        for every value assignment that needs to be explained. 
        The PQ is sorted using the size of the current hitting set of every 
        variable. Therefore, the element at the top of the PQ is the most likely
        to lead to an small explanation. 
        At every iteration of the split_ocus algorithm:
        
            1. The element at the top of the PQ is popped.
            2. One iteration in the SMUS algorithm is done.
                - Its hitting set cost is updated 
            3. The element is pushed by into the list 
        
        Args:
        - given: dict variable-value
        - vars_to_expl: collection of variables for which you want a potential
        explanation.
        - constraints: Reified-problem constraints
        

    """
    facts = [var == val for var, val in given.items()]
    assert Model(facts + constraints).solve(), "Model should be SAT!"
    sol = {var : var.value() for var in vars_to_expl}
    
    ## priority queue (PQ) of SMUS extractors for every value assignment
    ## that needs to be explained
    pq = [(var,0,SMUSAlgo(soft=facts + constraints, hard=[var != sol[var]])) 
          for var in vars_to_expl]
    
    i = 0
    while 1:
        var, obj_val, smus_algo = pq.pop(0)
        if verbose:
            print(f"\rContinuing computation on SMUS with obj {obj_val}", end="\t"*5)
        # pbar.set_description(f"Best objective_score: {obj_val}")
        if smus_algo.mus is not None:
            E = smus_algo.mus & set(facts)
            S = smus_algo.mus & set(constraints)
            N = [var == sol[var]]
            return (E,S,N)
        # Algo has not found a solution yet, continue
        smus_algo.iterate()
        pq.append((var,smus_algo.hs_solver.objective_value(),smus_algo))
        pq.sort(key=lambda x : x[1])

## La classe `EXPLANATION`

Considérons un problème de satisfaction de contraintes admettant une unique solution et que l'on peut modéliser par 3 données
- un tableau cpmpy *vars* de variables
- une liste *facts* de faits, contraintes de la forme $var == val$
- la liste *constraints* des autres contraintes.

Alors, après   

*explanation* = `EXPLANATION`(*vars*, *facts*, *constraints*)

*explanation*.solve() renvoie la solution sous la forme d'un tabeau numpy  
*explanation*.explain() renvoie une explication $(E,S,N)$  
*explanation*.explainFull() renvoie une liste d'explications $[(E_1,S_1,N_1),\ldots]$ permettant de résoudre tout le problème

In [ ]:
def print_explanation(E,S,N):

    print('\nFacts (E)')
    for e in E: print(f'   {e}')
    print('\nConstraints (S)')
    for c in S: print(f'   {c}')
    print('\n=> Hint (N)')
    print(f'   {N[0]}')


class EXPLANATION:

    def __init__(self, vars, facts, constraints):

        self.vars = vars                 # tableau cpmpy de variables
        self.facts = facts               # liste de "faits" de la forme var == value
        self.constraints = constraints   # liste de contraintes

        self.remaining_vars_to_expl = set(get_variables(constraints)) - set(get_variables(facts))

        assert Model(facts + constraints).solve(), "Model should be SAT!"
        self.clues = {var : var.value() for var in get_variables(facts)}  
        self.sol = {var : var.value() for var in get_variables(facts + constraints)} 

        self.explanation = None
        self.explanations = None

        sol = Model(self.facts + self.constraints).solve()
        assert sol, "Model is unsatisfiable."
        self.cpmpySol = self.vars.value()

    def solve(self):
        return self.cpmpySol

    def explain(self, verbose = True):
        """
        -> une explication (E, S, N)
        """

        if self.explanation is None: # sinon, inutile de recalculer self.explanation
            expl = split_ocus(self.clues, self.remaining_vars_to_expl, self.constraints)
            self.explanation = expl
            
        if verbose:
            print_explanation(*self.explanation)

        return self.explanation
            
    def explainFull(self, verbose = True):
        """
        -> les explicationS [(E_1,S_1,N_1), ...] pour la résolution complète
        """

        if self.explanations is None:
        
            expls = []
            
            while self.remaining_vars_to_expl:

                E, S, N = split_ocus(self.clues, self.remaining_vars_to_expl, self.constraints)
                var_N = get_variables(N)[0]
                self.clues[var_N] = self.sol[var_N]
                self.remaining_vars_to_expl -= set(get_variables(N))

                if verbose:
                    print_explanation(E, S, N)
                
                expls.append((E,S,N)) 
        
            self.explanations = expls
            self.explanation = expls[0]

        else: # inutile de recalculer self.explanations
            if verbose:
                for expl in self.explanations:
                    print_explanation(*expl)
            
        return self.explanations

## Outils

Les fonctions suivantes peuvent être utiles pour l'affichage des explications.

In [128]:
def extractVar(v):
    """
    v est une variable cpmpy
    extractVar(v)           -> v.name, si v n'est pas un élément d'un tableau
    extractVar(xxx[i])      -> i, de type int
    extractVar(xxx[i,j,..]) -> (i,j,..), tuple de int
    """
    c = v.name.split(',')
    n = len(c)
    if n == 0:
        return v.name 
    i1 = c[0].split('[')[1]
    if n == 1:
        return int(i1[:-1])
    i_n = int(c[-1].split(']')[0])
    return (int(i1),) + tuple([int(c[i]) for i in range(1,n-1)]) + (i_n,)

def extractFact(fact):
    """
    extractFact(var == value) -> extractVar(var), value 
    """
    return extractVar(fact.args[0]), int(fact.args[1])
   
def extractFacts(E):
    """
    extractFacts({fact_1, ...}) -> {extractFact(fact_1), ...)}
    """
    return set(map(extractFact,E))

def extractConstraint(constraint):
    """
    extractConstraint(xxx(var_1, ...)) -> [extractVar(var_1), ...]
    """
    return list(map(extractVar,constraint.args))

def extractConstraints(S):
    """
    extractConstraints({constraint_1, ...}) -> [extractConstraint(constraint_1), ...]
    """
    return list(map(extractConstraint,S))

def extractHint(N):
    """
    extractHint([var == value]) ->  extractVar(var), value 
    """
    assert len(N) == 1
    return extractFact(N[0])